In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [10]:
%run -i ../../zkstats/core.py

In [11]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [12]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('generated/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.sqrt(torch.var(dummy_data_tensor, correction = 1))
dummy_data_mean = torch.mean(dummy_data_tensor)

FileNotFoundError: [Errno 2] No such file or directory: 'generated/dummy_data.json'

In [ ]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*(X.size()[1]-1))<0.02*self.w*self.w*(X.size()[1]-1),x_mean_cons),self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [ ]:
# Prover/ data owner side
theory_output = torch.sqrt(torch.var(data_tensor, correction = 1))
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*self.w*(X.size()[1]-1))<0.02*self.w*self.w*(X.size()[1]-1),x_mean_cons),self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

theory output:  tensor(14.5800)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,1272],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1512,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [13]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.3459670543670654 seconds
Theory output:  tensor(14.5800)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 15.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [956231351009279921, 10951436676983309100, 2250248050743556928, 1228298028208591648]]], 'proof': '083ddbf380f29f43bbf4cea9af2860eba80a494e1a6f8ea28dad8305fe5f237e073595d5486706200376445624a9cc925fe2701e0829090037b0e19b9238c69c2e014a4cae1b302613f0123b719873e8eef7f3568ad6fa50dcddf62a0afb6bfd1856158aa9ef182eea35bc3ed577c0c67db5be90ba95a36d61bdb5b26a5940d717fc5ef1181f49686408671996e03a586c1924c62875576d2445a4f6e1a6e5e12cb21f737485ffd36cf9614f339ebb26117f9f133b35c5a1b874e38e087cef751640dff7b7406e3dbde0e6d94289dbc7ed6c170095daee15ffc98ff487a6b7070930c515bd6ae6797da4b8c78dafbb00f82a64898e106ba7bab74d531ca1af770e5fdf1e33df7b5031bb79d8ca1e76014b9f184734afef4785a12c70ef1c19fd2692de7a2b197d4a5fc7a1f58901c3a01e25cd4f467ca5894ea2c30e6e06c48a0434d61ebadbd560e2d71518aa75e0f6a9b271bf63654f8c735bbce390ab76

In [14]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [956231351009279921, 10951436676983309100, 2250248050743556928, 1228298028208591648]]]
proof boolean:  1.0
proof result 1 : 15.0
verified
